In [1]:
!git clone https://github.com/HoangHao1009/Chatbot
%cd Chatbot
!pip install -e .

Cloning into 'Chatbot'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 33 (delta 7), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (33/33), 14.61 KiB | 14.61 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/Chatbot
Obtaining file:///content/Chatbot
  Preparing metadata (setup.py) ... done
  Running setup.py develop for chatbot


In [2]:
from chatbot import Chatbot, Model, Trainer, Vocabulary
import torch
import torch.nn as nn

In [3]:
max_length = 15
vocandpair = Vocabulary.VocAndPairLoader()
vocandpair.PrepareDataFrom('/content/drive/Othercomputers/My Laptop/Personal/csvdata.txt', max_length)

Reading lines...


In [4]:
modeldata = Vocabulary.ModelData(vocandpair.voc, vocandpair.pairs)

In [5]:
modeldata.trimRareWords(2)

keep_words 18532 / 26986 = 0.6867
Trimmed from 111392 pairs to 104099, 0.9345 of total


In [6]:
hidden_size = 768
encoder_n_layers = 3
decoder_n_layers = 3
dropout = 0.2
batch_size = 64
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.device)
embedding = nn.Embedding(modeldata.voc.num_words, hidden_size)
encoder = Model.EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = Model.AttnDecoderRNN(embedding, hidden_size, modeldata.voc.num_words, decoder_n_layers, dropout)
encoder.to(device)
decoder.to(device)

<class 'torch.device'>


AttnDecoderRNN(
  (embedding): Embedding(18535, 768)
  (embedding_dropout): Dropout(p=0.2, inplace=False)
  (gru): GRU(768, 768, num_layers=3, dropout=0.2)
  (concat): Linear(in_features=1536, out_features=768, bias=True)
  (out): Linear(in_features=768, out_features=18535, bias=True)
  (attn): DotAttn()
)

In [13]:
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.00001
decoder_learning_ratio = 5.0
n_iteration = 2000
print_every = 10
save_every = 500
encoder.train()
decoder.train()

encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr = learning_rate)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr = learning_rate * decoder_learning_ratio)
training = Trainer.TrainIter(modeldata, encoder, decoder,
                             encoder_optimizer, decoder_optimizer,
                             embedding, n_iteration, batch_size, print_every, clip)
training.run(max_length, teacher_forcing_ratio)

Initializing...
Training...
Iteration: 10, Percent complete: 0.5, Avg loss: 3.0547257146269717
Iteration: 20, Percent complete: 1.0, Avg loss: 3.0323983057060584
Iteration: 30, Percent complete: 1.5, Avg loss: 3.051724480798054
Iteration: 40, Percent complete: 2.0, Avg loss: 3.0967251672909475
Iteration: 50, Percent complete: 2.5, Avg loss: 3.087882972351104
Iteration: 60, Percent complete: 3.0, Avg loss: 3.0868477563135808
Iteration: 70, Percent complete: 3.5000000000000004, Avg loss: 3.0242850760169886
Iteration: 80, Percent complete: 4.0, Avg loss: 3.0502219805112394
Iteration: 90, Percent complete: 4.5, Avg loss: 3.0061026368515655
Iteration: 100, Percent complete: 5.0, Avg loss: 3.077748278771616
Iteration: 110, Percent complete: 5.5, Avg loss: 3.1379200015516933
Iteration: 120, Percent complete: 6.0, Avg loss: 3.0006625382506718
Iteration: 130, Percent complete: 6.5, Avg loss: 3.0324428707477287
Iteration: 140, Percent complete: 7.000000000000001, Avg loss: 3.021139845508334
Iter

In [15]:
encoder.eval()
decoder.eval()

searcher = Chatbot.GreedySearchDecoder(encoder, decoder)
chatbot = Chatbot.Chatbot(searcher, modeldata.voc)
chatbot.chatting(15)

Me > hello
Bot: hello . . . . . . .
Me > how are you
Bot: i m fine . you know how i m a child .
Me > of course
Bot: and you have a hundred dollars ? ? ? ? ?
Me > no i'm not rich
Bot: no . . me . me . me .
Me > you are stupid
Bot: i m not a professional . you re a lot of things .
Me > i m not too
Bot: i m not going to be a minute . of your life .
Me > fuck you
Bot: i m sorry . you re a fucking fucking fucking person . !
Me > you are nasty
Bot: i m not a bit . you re a lot of things .
Me > go to hell
Bot: i m not going to be with you . of the business .
Me > q


In [16]:
torch.save({
    'en': encoder.state_dict(),
    'de': decoder.state_dict(),
    'en_opt': encoder_optimizer.state_dict(),
    'de_opt': decoder_optimizer.state_dict(),
    'voc_dict': vocandpair.voc.__dict__
}, '/content/drive/MyDrive/Chatbot/savemodel.pt')